# Competitors data

In [1]:
# Import libraries
import pandas as pd
import requests

In [2]:
# Look in documentation --> find place
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
api_key = "" # Your Google API key

# actual api key
location = "Zürich"
supermarkets = ["Lidl", "Aldi", "Denner", "Alnatura", "Spar", "Migros", "Coop"]
postal_codes = ["Schlieren", "Alstetten", "Weningen", "Unterengstringen", "Uitikon",
                "Albisrieden", "Afforltern", "Wallisellen", "Hochbord"]

search_terms = []

for supermarket in supermarkets:
    
    for postal_code in postal_codes:
        
        search_term = supermarket + ", " + location + ", " + str(postal_code)
        
        search_terms.append(search_term)
    
search_terms

['Lidl, Zürich, Schlieren',
 'Lidl, Zürich, Alstetten',
 'Lidl, Zürich, Weningen',
 'Lidl, Zürich, Unterengstringen',
 'Lidl, Zürich, Uitikon',
 'Lidl, Zürich, Albisrieden',
 'Lidl, Zürich, Afforltern',
 'Lidl, Zürich, Wallisellen',
 'Lidl, Zürich, Hochbord',
 'Aldi, Zürich, Schlieren',
 'Aldi, Zürich, Alstetten',
 'Aldi, Zürich, Weningen',
 'Aldi, Zürich, Unterengstringen',
 'Aldi, Zürich, Uitikon',
 'Aldi, Zürich, Albisrieden',
 'Aldi, Zürich, Afforltern',
 'Aldi, Zürich, Wallisellen',
 'Aldi, Zürich, Hochbord',
 'Denner, Zürich, Schlieren',
 'Denner, Zürich, Alstetten',
 'Denner, Zürich, Weningen',
 'Denner, Zürich, Unterengstringen',
 'Denner, Zürich, Uitikon',
 'Denner, Zürich, Albisrieden',
 'Denner, Zürich, Afforltern',
 'Denner, Zürich, Wallisellen',
 'Denner, Zürich, Hochbord',
 'Alnatura, Zürich, Schlieren',
 'Alnatura, Zürich, Alstetten',
 'Alnatura, Zürich, Weningen',
 'Alnatura, Zürich, Unterengstringen',
 'Alnatura, Zürich, Uitikon',
 'Alnatura, Zürich, Albisrieden',
 'Al

In [3]:
competitors_df = pd.DataFrame()

for search_term in search_terms:
    
    query = search_term
    
    next_page_token = ""
    
    for num_page in range(1, 4):
    
        try:
            req = requests.get(url + "query=" + query + "&key=" + api_key + "&next_page_token=" + "")
        except:
            print("Error in the get")
    
        
        places_json = req.json()
    
        try:
            next_page_token = places_json["next_page_token"]
        except:
            next_page_token = ""
        
        
        results_df = pd.json_normalize(places_json["results"])
        results_df["supermarket_brand"] = search_term
        competitors_df = competitors_df.append(results_df)
        
    competitors_df = competitors_df.drop_duplicates(["formatted_address", "name", "supermarket_brand"]).reset_index(drop=True)
        
    print("{}: OK".format(search_term))

Lidl, Zürich, Schlieren: OK
Lidl, Zürich, Alstetten: OK
Lidl, Zürich, Weningen: OK
Lidl, Zürich, Unterengstringen: OK
Lidl, Zürich, Uitikon: OK
Lidl, Zürich, Albisrieden: OK
Lidl, Zürich, Afforltern: OK
Lidl, Zürich, Wallisellen: OK
Lidl, Zürich, Hochbord: OK
Aldi, Zürich, Schlieren: OK
Aldi, Zürich, Alstetten: OK
Aldi, Zürich, Weningen: OK
Aldi, Zürich, Unterengstringen: OK
Aldi, Zürich, Uitikon: OK
Aldi, Zürich, Albisrieden: OK
Aldi, Zürich, Afforltern: OK
Aldi, Zürich, Wallisellen: OK
Aldi, Zürich, Hochbord: OK
Denner, Zürich, Schlieren: OK
Denner, Zürich, Alstetten: OK
Denner, Zürich, Weningen: OK
Denner, Zürich, Unterengstringen: OK
Denner, Zürich, Uitikon: OK
Denner, Zürich, Albisrieden: OK
Denner, Zürich, Afforltern: OK
Denner, Zürich, Wallisellen: OK
Denner, Zürich, Hochbord: OK
Alnatura, Zürich, Schlieren: OK
Alnatura, Zürich, Alstetten: OK
Alnatura, Zürich, Weningen: OK
Alnatura, Zürich, Unterengstringen: OK
Alnatura, Zürich, Uitikon: OK
Alnatura, Zürich, Albisrieden: OK
Alna

In [4]:
print(competitors_df.shape)

(862, 24)


In [5]:
competitors_df.head()

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,...,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,supermarket_brand,price_level,permanently_closed
0,OPERATIONAL,"Reitmenstrasse 5, 8952 Schlieren, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl,"[{'height': 2260, 'html_attributions': ['<a hr...",ChIJLWHT12sMkEcRmSuzLQv5y5k,4.1,ChIJLWHT12sMkEcRmSuzLQv5y5k,...,47.400842,8.434019,47.398142,8.431319,True,9CXM+P2 Schlieren,8FVC9CXM+P2,"Lidl, Zürich, Schlieren",NaN,NaN
1,OPERATIONAL,"Förrlibuckstrasse 62, 8005 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 1928, 'html_attributions': ['<a hr...",ChIJyzfNMz8LkEcRQoUMrhNYjOA,4.3,ChIJyzfNMz8LkEcRQoUMrhNYjOA,...,47.393899,8.517091,47.391199,8.514391,True,9GR8+X7 Zürich,8FVC9GR8+X7,"Lidl, Zürich, Alstetten",NaN,NaN
2,OPERATIONAL,"Wehntalerstrasse 625, 8046 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 1960, 'html_attributions': ['<a hr...",ChIJseGe3eQLkEcR4zpsLTAFO8k,4.2,ChIJseGe3eQLkEcR4zpsLTAFO8k,...,47.423829,8.500618,47.421130,8.497918,True,CFCX+XM Zürich,8FVCCFCX+XM,"Lidl, Zürich, Alstetten",NaN,NaN
3,OPERATIONAL,"Grubenstrasse 28, 8045 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 540, 'html_attributions': ['<a hre...",ChIJbRRqLvMJkEcR_mqMiGkBWaw,4.1,ChIJbRRqLvMJkEcR_mqMiGkBWaw,...,47.363033,8.516097,47.360333,8.513397,True,9G67+PV Zürich,8FVC9G67+PV,"Lidl, Zürich, Alstetten",NaN,NaN
4,OPERATIONAL,"Fraumünsterstrasse 16, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,LIDL Switzerland,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ0REDgqqgmkcRWrTSmPTdg1Q,4.4,ChIJ0REDgqqgmkcRWrTSmPTdg1Q,...,47.370288,8.542742,47.367589,8.540042,True,9G9R+GF Zürich,8FVC9G9R+GF,"Lidl, Zürich, Alstetten",NaN,NaN


In [6]:
# Save data
competitors_df.to_csv("C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/supermarkets_2_raw_data.csv", index=False)

# Data preparation

In [24]:
# Read data
competitors_df_1 = pd.read_csv("C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/supermarkets_raw_data.csv")
competitors_df_2 = pd.read_csv("C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/supermarkets_2_raw_data.csv")

# Merge dataframes
competitors_df = pd.concat([competitors_df_1, competitors_df_2]).reset_index(drop=True)
competitors_df.head(5)

,business_status,formatted_address,icon,icon_background_color,icon_mask_base_uri,name,photos,place_id,rating,reference,...,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,opening_hours.open_now,plus_code.compound_code,plus_code.global_code,supermarket_brand,price_level,permanently_closed
0,OPERATIONAL,"Fraumünsterstrasse 16, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,LIDL Switzerland,"[{'height': 540, 'html_attributions': ['<a hre...",ChIJ0REDgqqgmkcRWrTSmPTdg1Q,4.4,ChIJ0REDgqqgmkcRWrTSmPTdg1Q,...,47.370288,8.542742,47.367589,8.540042,True,9G9R+GF Zürich,8FVC9G9R+GF,"Lidl, Zürich, 8001",NaN,NaN
1,OPERATIONAL,"Fraumünsterstrasse 16, 8001 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,LIDL Switzerland,"[{'height': 2736, 'html_attributions': ['<a hr...",ChIJ0REDgqqgmkcRWrTSmPTdg1Q,4.4,ChIJ0REDgqqgmkcRWrTSmPTdg1Q,...,47.370288,8.542742,47.367589,8.540042,True,9G9R+GF Zürich,8FVC9G9R+GF,"Lidl, Zürich, 8002",NaN,NaN
2,OPERATIONAL,"Grubenstrasse 28, 8045 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJbRRqLvMJkEcR_mqMiGkBWaw,4.1,ChIJbRRqLvMJkEcR_mqMiGkBWaw,...,47.363033,8.516097,47.360333,8.513397,True,9G67+PV Zürich,8FVC9G67+PV,"Lidl, Zürich, 8002",NaN,NaN
3,OPERATIONAL,"Förrlibuckstrasse 62, 8005 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 1928, 'html_attributions': ['<a hr...",ChIJyzfNMz8LkEcRQoUMrhNYjOA,4.3,ChIJyzfNMz8LkEcRQoUMrhNYjOA,...,47.393540,8.516621,47.390840,8.513921,True,9GR8+X7 Zürich,8FVC9GR8+X7,"Lidl, Zürich, 8002",NaN,NaN
4,OPERATIONAL,"Ohmstrasse 11, 8050 Zürich, Switzerland",https://maps.gstatic.com/mapfiles/place_api/ic...,#4B96F3,https://maps.gstatic.com/mapfiles/place_api/ic...,Lidl Schweiz,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJU0W-noYKkEcRXJuSYfWbBR8,4.1,ChIJU0W-noYKkEcRXJuSYfWbBR8,...,47.411778,8.547188,47.409079,8.544488,True,CG6W+57 Zürich,8FVCCG6W+57,"Lidl, Zürich, 8002",NaN,NaN


In [25]:
# Filter the supermarkets that are "permanently_closed"
competitors_df = competitors_df[competitors_df["permanently_closed"] != True]

# Split column supermarket_brand
competitors_df[["search_name", "search_location", "search_postal_code"]] = competitors_df["supermarket_brand"].str.split(",", expand=True)

# Columns selection
competitors_df = competitors_df[["geometry.viewport.northeast.lat", "geometry.viewport.northeast.lng", "formatted_address", "name", "rating", "search_name", "search_postal_code"]]

# Check for outliers names
# print(competitors_df["name"].unique())

names_filter = ['Billett-Service Migros', 'SUN STORE Migros City Zürich', 'Migros Florissimo', 'VOI Migros Partner','Migros-Genossenschafts-Bund Group IT',
                 'Do it + Garden - City Farming Zürich-Oerlikon', 'melectronics', 'Migros Electronics', 'VOI', 'REFORMHAUS', 'DHL Service Point (SPAR EXPRESS)', 'Postal agency 8068 Zürich West',
                 'DHL Service Point (Spar Supermarkt Zollfreilager)', 'Dürr A. & CO AG', 'Klubschule Migros Zürich Oerlikon', 'Do it + Garden Zurich - Brunaupark', 'SportXX', 'Neumarkt Oerlikon',
                 'Coop Bau+Hobby Zürich Oerlikon Eleven', 'Reto Suhner', 'REFORMHAUS', 'Maxi Frischmarkt Zürich', 'Billett-Service Migros', 'Do it + Garden - City Farming Zürich-Oerlikon',
                 'Do it + Garden Zurich - Brunaupark', 'Migros Florissimo Glattzentrum', 'Chicorée', 'Migros Bank', 'Micasa', 'Reto Suhner', 'Coop Vitality Zürich Sihl City']

# Filter outliers
competitors_df = competitors_df[~competitors_df.name.isin(names_filter)]

# Change name to upper
competitors_df["name"] = competitors_df["name"].str.upper()
competitors_df["search_name"] = competitors_df["search_name"].str.upper()

competitors_df = competitors_df[competitors_df.apply(lambda x: x['search_name'] in x['name'], axis = 1)]
competitors_df = competitors_df.drop_duplicates(["formatted_address", "name", "search_name"]).reset_index(drop=True)

competitors_df["name"].unique()

array(['LIDL SWITZERLAND', 'LIDL SCHWEIZ', 'LIDL', 'SUPER MARKET LIDL',
       'ALDI', 'ALDI ZÜRICH', 'ALDI SUISSE', 'ALDI SUISSE - ZURICH',
       'ALDI 2', 'ALDI SUISSE AG', 'ALDI SCHLIEREN', 'ALDI ZUMIKON',
       'ALDI ADLISWIL', 'ALDI DÜBENDORF', 'ALDI REGENSDORF',
       'ALDI RÜMLANG', 'ALDI DIETIKON', 'ALDI MEILEN',
       'ALDI AFFOLTERN A. A.', 'ALDI DIELSDORF', 'DENNER EXPRESS',
       'DENNER DISCOUNT', 'DENNER PARTNER', 'DENNER',
       'ALNATURA BIO SUPER MARKT', 'SPAR EXPRESS ZÜRICH SCHAUSPIELHAUS',
       'SPAR SUPERMARKT DOLDERSTRASSE', 'SPAR EXPRESS ZÜRICH TOBLERPLATZ',
       'SPAR SUPERMARKT ZÜRICH FORCHSTRASSE',
       'SPAR EXPRESS ZÜRICH HARDTURMSTRASSE',
       'SPAR SUPERMARKT GERLAFINGEN', 'EUROSPAR', 'SPAR SUPERMARKT',
       'SPAR SUPERMARKT ZOLLFREILAGER', 'SPAR SUPERMARKT SCHLIEREN',
       'SPAR SUPERMARKT DÜBENDORF', 'SPAR SUPERMARKT RÜSCHLIKON',
       'SPAR SUPERMARKT BRAMEN', 'SPAR SUPERMARKT KLOTEN BREITISTRASSE',
       'SPAR SUPERMARKT DIETLIKON', 

In [28]:
competitors_df.shape

(343, 7)

In [29]:
# Save data
competitors_df.to_csv("C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/supermarkets_clean_data.csv", index=False)

# Data Assignation

In [35]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point
from kd_tree import ckdnearest

ModuleNotFoundError: No module named 'geopandas'

In [33]:
# remove unneccessary columns from raw data
df = pd.read_csv('C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/GeoFeatures_Zurich_provided_by_UrbanDataLabs.csv')
df_dropped = df[[ 'lat', 'lng', 'hh_ha', 'pers_ha', 'pt_dis', 'station_dis', 'bldg_foot']]
df_dropped.to_csv('C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/GeoFeatures_Zurich_needed.csv', index=False)
df_dropped.head()

,lat,lng,hh_ha,pers_ha,pt_dis,station_dis,bldg_foot
0,47.3631,8.53425,8,10,55,1725,4204
1,47.3634,8.53425,53,100,50,1700,4204
2,47.3627,8.53457,8,10,75,1770,4204
3,47.3629,8.53457,8,10,55,1745,4204
4,47.3631,8.53458,8,10,35,1720,4204


In [34]:
# add new column with both coordinates as type point in dataframes that need to be compared for closest neighbours
df_geofeatures = pd.read_csv('C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/GeoFeatures_Zurich_needed.csv')
df_geofeatures['geometry'] = [Point(df_geofeatures['lat'][i], df_geofeatures['lng'][i]) for i in range(len(df_geofeatures['lat']))]
df_geofeatures.head(5)


NameError: name 'Point' is not defined

In [ ]:
df_competitors = pd.read_csv('C:/Users/RCOLL/OneDrive/Documents/GitHub/migros_data_challange/data/supermarkets_clean_data.csv')
df_competitors = df_competitors.rename(columns={'geometry.viewport.northeast.lat': 'lat_shop', 'geometry.viewport.northeast.lng': 'lng_shop'})
df_competitors['geometry'] = [Point(df_competitors['lat_shop'][i], df_competitors['lng_shop'][i]) for i in range(len(df_competitors['lat_shop']))]

In [ ]:
# calculate nearest neighbour
assigned_df = ckdnearest(df_competitors, df_geofeatures)

# make df with number of supermarkets for each coordinate point in original geofeatures file
# need to get rid of geometry in one df so it won't be in the new df twice
assigned_df = assigned_df.drop('geometry', axis=1)

#new column to compare closest coordinates of supermarkets and take them together
assigned_df['lat, lng'] = assigned_df['lat'].astype(str) + ', ' + assigned_df['lng'].astype(str)

#new df to drop duplicates of coordinates
new_df = assigned_df[['lat, lng', 'lat', 'lng']]
new_df = new_df.drop_duplicates('lat, lng')
list_lat_lng = new_df['lat, lng'].tolist()

#make lists of new columns wanted in the new df to keep count of supermarkets per point
migros, coop, discounter, other = [], [], [], []
sup_columns = [migros, coop, discounter, other]
distance_limit = 1 #to make sure no supermarket outside of our region of interest is accidently included

for i in range(len(list_lat_lng)):
    
    lat_lng = list_lat_lng[i]
    
    for k in range(len(sup_columns)):
        sup_columns[k].append(0)
    df_same = assigned_df[assigned_df['lat, lng'] == lat_lng]
    count = df_same['search_name'].count()
    
    for c in range(count):
        
        distance = df_same.iloc[c].dist
        
        if distance > distance_limit:
            continue
        brand = df_same.iloc[c].search_name.lower()
        
        if brand in ['migros']:
            migros[i] += 1
        
        elif brand in ['coop']:
            coop[i] += 1
        
        elif brand in ['lidl', 'aldi']:
            discounter[i] += 1
        
        elif brand in ['denner']: #exclude denner since next to nearly every migros is a denner and they are the same
            continue
    
        else:
            other[i] += 1

df = pd.DataFrame({'lat': new_df['lat'].tolist(),
                   'lng': new_df['lng'].tolist(),
                   'migros': migros,
                   'coop': coop,
                   'discounter': discounter,
                   'other': other,})

df['competitors'] = df['coop'] + df['discounter'] + df['other'] #count all competitor supermarkets per coordinate

df['all_supermarkets'] = df['migros'] + df['coop'] + df['discounter'] + df['other'] #count all supermarkets per coordinate

df_merged = pd.merge(df_geofeatures, df, how='outer')

df_merged = df_merged.fillna(0)

df_merged.to_csv('C:/Users/Martina/Documents/Propulsion/migros_data_challange/data/GeoFeatures_Zurich_and_supermarkets.csv', index=False)